In [8]:
import zipfile
import os

# Path to the zip file
zip_file = r"C:\Users\abhay\Computer vision\comp5623m-artificial-intelligence.zip"

# Destination folder (same directory as the script)
extract_to = r"C:\Users\abhay\Computer vision"

# Create the destination folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")

Files extracted to: C:\Users\abhay\Computer vision


In [1]:
import math
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader,Dataset
from torch.hub import load_state_dict_from_url
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn.metrics import confusion_matrix, roc_curve, auc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Class for loading datasets
class CustomImageDataset(torch.utils.data.Dataset):
    
    # Initialize the dataset with image directory and transform
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {class_name: i for i, class_name in enumerate(self.class_names)}
        self.images = []
        self.labels = []
        for class_name in self.class_names:
            class_dir = os.path.join(root_dir, class_name)
            for filename in os.listdir(class_dir):
                if filename.endswith('.JPEG'):
                    self.images.append(os.path.join(class_dir, filename))
                    self.labels.append(self.class_to_idx[class_name])

    # This function returns the number of samples in the dataset.   
    def __len__(self):
        return len(self.images)
    
    # This function returns the image and the label index.
    def __getitem__(self, index):
        image_path = self.images[index]
        label = self.labels[index]
        image = torchvision.datasets.folder.default_loader(image_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [3]:
def dataloading(transform):
    # Load the dataset of images and apply the transformation
    dataset = CustomImageDataset(root_dir='/kaggle/input/train-dataset/train_set/train_set')
    
    valid_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    # Set the random seed for reproducibility
    torch.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    
    # Splitting the dataset for validation and training in an 80:20 ratio     
    training_size = int(0.8 * len(dataset))
    validation_size = len(dataset) - training_size
    training_dataset, validation_dataset = torch.utils.data.random_split(dataset, [training_size, validation_size])
    
    training_dataset.dataset.transform = transform
    validation_dataset.dataset.transform = valid_transform

    # Create a DataLoader for the training set and validation set
    training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=16, num_workers=2, shuffle=True, generator=torch.Generator().manual_seed(0))
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=16, num_workers=2, shuffle=False)

    # Get the class names (labels) corresponding to the subfolders in the dataset
    class_names = dataset.class_names
    
    # Return the datasets and dataloaders for training and validation, along with the class names
    return training_dataset, validation_dataset, training_dataloader, validation_dataloader, class_names

In [4]:
# Define the image preprocessing steps
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load the training and validation datasets, and create dataloaders for them
training_dataset, validation_dataset, training_dataloader, validation_dataloader, class_names = dataloading(train_transform)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/train-dataset/train_set/train_set'